# Análise de Séries Temporais do Saldo em juros flutuantes outros de depósitos a prazo (CDB/RDB) - Pessoas físicas

In [7]:
pip install arch


   ---------------------------------------- 0.0/924.9 kB ? eta -:--:--
   ---- ----------------------------------- 112.6/924.9 kB 3.3 MB/s eta 0:00:01
   ---------------- ----------------------- 389.1/924.9 kB 4.9 MB/s eta 0:00:01
   ------------------------------ --------- 716.8/924.9 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 924.9/924.9 kB 5.9 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from arch import arch_model


### Leitura e Visualização dos dados

In [11]:
# Carregar a série temporal
data = pd.read_csv('STP-20240926172728724.csv', parse_dates=['Data'], index_col='Data')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 51: invalid continuation byte

In [9]:

# Visualização inicial
plt.figure(figsize=(12,6))
plt.plot(data, label='Saldo em Juros Flutuantes CDB/RDB - Pessoas Físicas')
plt.title('Série Temporal: Saldo em Juros Flutuantes CDB/RDB - Pessoas Físicas')
plt.xlabel('Data')
plt.ylabel('Saldo (Milhares de Reais)')
plt.legend()
plt.show()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 51: invalid continuation byte

### Decomposição da Série Temporal

In [ ]:
# Decomposição da série
decomposition = sm.tsa.seasonal_decompose(data, model='additive')

# Plotando decomposição
fig = decomposition.plot()
fig.set_size_inches(10, 8)
plt.show()


### Correlograma e PACF com Intervalo de Confiança


Correlograma para autocorrelação (ACF) e autocorrelação parcial (PACF) com intervalos de confiança de 95%:

In [ ]:
# Correlograma ACF
plot_acf(data, lags=40)
plt.title('Correlograma - ACF')
plt.show()

# Correlograma PACF
plot_pacf(data, lags=40)
plt.title('Correlograma - PACF')
plt.show()


### Teste de Estacionariedade (ADF Test)

Aplicamos o teste de Dickey-Fuller para verificar se a série é estacionária:

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Teste ADF
result = adfuller(data)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')


Se o p-valor for menor que 0.05, a série é estacionária, caso contrário, podemos aplicar uma diferenciação:

In [ ]:
# Diferenciando os dados
data_diff = data.diff().dropna()

# Plotando a série diferenciada
plt.figure(figsize=(12,6))
plt.plot(data_diff, label='Série Diferenciada')
plt.title('Série Temporal Diferenciada')
plt.xlabel('Data')
plt.ylabel('Diferença do Saldo')
plt.legend()
plt.show()


###  Modelo Auto ARIMA

Agora utilizamos o auto_arima para selecionar o melhor modelo ARIMA automaticamente com base em AIC:

In [ ]:
# Modelo Auto ARIMA
auto_arima_model = auto_arima(data, seasonal=False, trace=True,
                              error_action='ignore', suppress_warnings=True, stepwise=True)

# Resumo do modelo
print(auto_arima_model.summary())

# Ajuste do modelo ARIMA
model_fit = auto_arima_model.fit(data)

# Previsões
forecast = model_fit.predict(n_periods=30)
plt.figure(figsize=(12,6))
plt.plot(data, label='Série Original')
plt.plot(pd.date_range(data.index[-1], periods=30, freq='D'), forecast, label='Previsão', color='red')
plt.title('Previsão com Auto ARIMA')
plt.xlabel('Data')
plt.ylabel('Saldo')
plt.legend()
plt.show()


### Modelo GARCH
O modelo GARCH é adequado para capturar a volatilidade na série temporal, que pode ser comum em séries financeiras:

In [ ]:
# Modelo GARCH
garch_model = arch_model(data_diff, vol='Garch', p=1, q=1)
garch_fit = garch_model.fit()
print(garch_fit.summary())

# Previsão da volatilidade
garch_forecast = garch_fit.forecast(horizon=30)
plt.figure(figsize=(12,6))
plt.plot(garch_forecast.variance[-1:], label='Volatilidade Prevista')
plt.title('Previsão de Volatilidade com GARCH')
plt.xlabel('Data')
plt.ylabel('Volatilidade')
plt.legend()
plt.show()


### Avaliação do Modelo com Cross-Validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Validação cruzada para séries temporais
tscv = TimeSeriesSplit(n_splits=5)
errors = []

for train_index, test_index in tscv.split(data):
    train, test = data.iloc[train_index], data.iloc[test_index]
    model = auto_arima(train, seasonal=False, stepwise=True)
    predictions = model.predict(n_periods=len(test))
    error = mean_squared_error(test, predictions)
    errors.append(error)

print(f'Mean Squared Error: {np.mean(errors)}')
